In [1]:
import ROOT
import numpy as np

In [72]:
f1 = ROOT.TFile.Open('SURPRISE_Test_Samples_v10_04_07_05_Run4b_hyper_unified_reco2_BNB_nu_overlay_may8_reco2_hist_62280499_snapshot.root')
t1 = f1.Get('nuselection/NeutrinoSelectionFilter')


In [73]:
df = ROOT.RDataFrame(t1)
df.GetColumnNames()

vector<string>{ "CosmicDirAll2DEnds", "CosmicDirAll2DOvlp", "CosmicDirAll3D", "CosmicIP", "CosmicIPAll2DEnds", "CosmicIPAll2DOvlp", "CosmicIPAll3D", "CylFrac1h_1cm", "CylFrac1h_2cm", "CylFrac1h_3cm", "CylFrac1h_4cm", "CylFrac1h_5cm", "CylFrac2h_1cm", "CylFrac2h_2cm", "CylFrac2h_3cm", "CylFrac2h_4cm", "CylFrac2h_5cm", "CylFrac_1cm", "CylFrac_2cm", "CylFrac_3cm", "CylFrac_4cm", "CylFrac_5cm", "DeltaMed", "DeltaMed1h", "DeltaMed2h", "DeltaRMS", "DeltaRMS1h", "DeltaRMS2h", "Med_TT3", "NeutrinoEnergy0", "NeutrinoEnergy1", "NeutrinoEnergy2", "PCAWin_1Cr_2_5cm", "PCAWin_1Cr_5cm", "PCAWin_2Cr_2_5cm", "PCAWin_2Cr_5cm", "PCAWin_3Cr_2_5cm", "PCAWin_3Cr_5cm", "PCAWin_dist_2_5cm", "PCAWin_dist_5cm", "PCAWin_npts_2_5cm", "PCAWin_npts_5cm", "RootinoFix", "SliceCaloEnergy0", "SliceCaloEnergy1", "SliceCaloEnergy2", "X_SpcPts_v", "Y_SpcPts_v", "Z_SpcPts_v", "_closestNuCosmicDist", "_elecclusters_U_N", "_elecclusters_U_charge", "_elecclusters_V_N", "_elecclusters_V_charge", "_elecclusters_Y_N", "_elecclu

In [4]:
ary = df.AsNumpy(['trk_energy_proton_v'])
len(ary['trk_energy_proton_v'])

459154

In [5]:
print(np.sqrt(0.06213238 **2 + 0.16997787 **2 + 1.9867662e+00**2))

1.994991940434006


In [6]:
%%cpp
# include <cmath>

float magnitude(float x, float y, float z){
        return sqrt(x*x + y*y + z*z);
    }

In [7]:
%%cpp
# include <cmath>

// Last updated on 23 June 2021 by Steven Gardiner .
// Returns true if a given PDG code represents a meson or antimeson . Otherwise
// returns false . Based on points 10 , 12 , and 13 of the Particle Data Group ’s
// " Monte Carlo Particle Numbering Scheme " (2019 revision ).
bool is_meson ( int pdg_code ) {

// Ignore differences between mesons and antimesons for this test . Mesons
// will have positive PDG codes , while antimesons will have negative ones .
    int abs_pdg = std :: abs ( pdg_code ) ;

// Meson PDG codes have no more than seven digits . Seven - digit
// codes beginning with "99" are reserved for generator - specific
// particles
    if ( abs_pdg >= 9900000 ) return false ;

// Mesons have a value of zero for $n_ {q1}$, the thousands digit
    int thousands_digit = ( abs_pdg / 1000 ) % 10;
    if ( thousands_digit != 0 ) return false ;

// They also have a nonzero value for $n_ {q2}$, the hundreds digit
    int hundreds_digit = ( abs_pdg / 100 ) % 10;
    if ( hundreds_digit == 0 ) return false ;

// Reserved codes for Standard Model parton distribution functions
    if ( abs_pdg >= 901 && abs_pdg <= 930 ) return false ;

    // Reggeon and pomeron
    if ( abs_pdg == 110 || abs_pdg == 990 ) return false ;

// Reserved codes for GEANT tracking purposes
    if ( abs_pdg == 998 || abs_pdg == 999 ) return false ;

// Reserved code for generator - specific pseudoparticles
    if ( abs_pdg == 100 ) return false ;

// If we ’ve passed all of the tests above , then the particle is a meson
    return true ;
}

In [8]:
%%cpp
double rel_mom(float mass, float nrg){
    double p_reco = std::sqrt(nrg*nrg + 2. * nrg * mass);
    return p_reco;
}

In [9]:
%%cpp

int is_Signal(Float_t true_nu_vtx_x, Float_t true_nu_vtx_y, Float_t true_nu_vtx_z, Int_t nu_pdg,
    Int_t ccnc, ROOT::VecOps::RVec<int> mc_pdg, 
        ROOT::VecOps::RVec<float> mc_px, ROOT::VecOps::RVec<float> mc_py, ROOT::VecOps::RVec<float> mc_pz){
    
    
    
    bool isSignal = true;
    if (true_nu_vtx_x > 234.85 || true_nu_vtx_x < 21.5){
        isSignal = false;
    }
    if (true_nu_vtx_y > 95.00 || true_nu_vtx_y < -95.00){
        isSignal = false;
    }
    if (true_nu_vtx_z > 966.8 || true_nu_vtx_z < 21.5){
       isSignal = false;
    }
    if (ccnc != 0 || nu_pdg != 14){
        isSignal = false;
    }
    bool isProton = false;
    for (size_t i = 0; i < mc_pdg.size(); ++i){
        if (mc_pdg[i] == 2212){
            isProton = true;
            break;
        }    
    }
    if (!(isProton)){
        isSignal = false;
    }
    
    bool muon_in_mom = false;
    for (size_t i = 0; i < mc_pdg.size(); ++i){
        if (mc_pdg[i] != 13) continue;
        
        float mag = magnitude(mc_px[i], mc_py[i], mc_pz[i]);
        
        if (mag <= 1.2 && mag >= 0.1){
           muon_in_mom = true;
            
        }
        
    }
    if (!(muon_in_mom)){
            isSignal = false;
        }
    
    float leading = -999.0;
    
    for (size_t i = 0; i < mc_px.size(); ++i){
        if (mc_pdg[i] != 2212) continue;
        
        float mag = magnitude(mc_px[i], mc_py[i], mc_pz[i]);
        
        if (mag > leading){
            leading = mag;
        }
        
    }
    if (leading > 1.0 || leading < 0.25){
            isSignal = false;
        }
    

    
    int mesons = 0;
    for (size_t i = 0; i < mc_pdg.size(); ++i){
        if (is_meson(mc_pdg[i])){
            mesons += 1;
            break;
        }
    }
    if (mesons != 0){
        isSignal = false;
    }
    if (isSignal){
        return 1;
    }
    else{
        return 0;
    }
}

In [10]:
%%cpp
int signal_1(Float_t true_nu_vtx_x, Float_t true_nu_vtx_y, Float_t true_nu_vtx_z){
    int passfail = 1;
    if (true_nu_vtx_x > 234.85 || true_nu_vtx_x < 21.5){
        passfail = 0;
    }
    if (true_nu_vtx_y > 95.00 || true_nu_vtx_y < -95.00){
        passfail = 0;
    }
    if (true_nu_vtx_z > 966.8 || true_nu_vtx_z < 21.5){
       passfail = 0;
    }
    return passfail;
}

In [11]:
%%cpp
int signal_2(Int_t ccnc){
    int passfail = 1;
    if (ccnc != 0){
        passfail = 0;
    }
    return passfail;
}

In [12]:
%%cpp
int signal_3(ROOT::VecOps::RVec<int> mc_pdg){
    int passfail = 1;
    bool isProton = false;
    for (size_t i = 0; i < mc_pdg.size(); ++i){
        if (mc_pdg[i] == 2212){
            isProton = true;
            break;
        }    
    }
    if (!(isProton)){
        passfail = 0;
    }
    return passfail;
}

In [13]:
%%cpp
int signal_4(ROOT::VecOps::RVec<int> mc_pdg, 
    ROOT::VecOps::RVec<float> mc_px, ROOT::VecOps::RVec<float> mc_py, ROOT::VecOps::RVec<float> mc_pz){
        int passfail = 1;
        for (size_t i = 0; i < mc_px.size(); ++i){
            if (mc_pdg[i] != 13) continue;
        
            float mag = magnitude(mc_px[i], mc_py[i], mc_pz[i]);
        
            if (mag > 1.2 || mag < 0.1){
               passfail = 0;
            }
        }
        return passfail;
}

In [14]:
%%cpp
int signal_5(ROOT::VecOps::RVec<int> mc_pdg, 
    ROOT::VecOps::RVec<float> mc_px, ROOT::VecOps::RVec<float> mc_py, ROOT::VecOps::RVec<float> mc_pz){
        int passfail = 1;
        float leading = magnitude(mc_px[0], mc_py[0], mc_pz[0]);
    
        for (size_t i = 0; i < mc_px.size(); ++i){
            if (mc_pdg[i] != 2212) continue;
        
            float mag = magnitude(mc_px[i], mc_py[i], mc_pz[i]);
        
            if (mag > leading){
                leading = mag;
            }
        
        }
        if (leading > 1.0 || leading < 0.25){
                passfail = 0;
        } 
    return passfail;
}
        

In [15]:
%%cpp 
int signal_6(ROOT::VecOps::RVec<int> mc_pdg){
    int passfail = 1;
    int mesons = 0;
    for (size_t i = 0; i < mc_pdg.size(); ++i){
        if (is_meson(mc_pdg[i])){
            mesons += 1;
            break;
        }
    }
    if (mesons != 0){
        passfail = 0;
    }
    return passfail;
}

In [16]:
## now for the selection criteria

In [17]:
%%cpp
int muon_ID(ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<float> trk_len_v){
    ROOT::VecOps::RVec<int> muon_indices;
    ROOT::VecOps::RVec<double> muon_scores;
    for (size_t i = 0; i < trk_llr_pid_score_v.size(); ++i){
            if (trk_score_v.at(i) > 0.8 && trk_distance_v.at(i) < 4 && trk_len_v.at(i) > 10 && trk_llr_pid_score_v.at(i) > 0.2){
                muon_indices.push_back(i);
                muon_scores.push_back(trk_llr_pid_score_v.at(i));
                
            }
        }
    int muon_i = -999;
    
    if (muon_indices.size() == 0){
        muon_i = -1;
        return muon_i;
    }
    double muon_score = -99.0;
    if (muon_indices.size() > 1){
        for (size_t i = 0; i < muon_scores.size(); ++i){
            if (muon_scores.at(i) > muon_score) {
                muon_score = muon_scores.at(i);
                muon_i = muon_indices.at(i);
            }
        }
    }
    else{
        muon_i = muon_indices.at(0);
    }
    return muon_i;
        
}

In [18]:
%%cpp
int LeadingID(ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<float> trk_len_v,
             ROOT::VecOps::RVec<float> trk_energy_proton_v, ROOT::VecOps::RVec<int> generation_v){
    int leading = -1;
    vector<int> proton_candidates;
    int muon_i = muon_ID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v);
    for (size_t i = 0; i < trk_score_v.size(); ++i){
        if (generation_v[i] != 2) continue;
        if (i == muon_i) continue;
        if (trk_score_v[i] < 0.5) continue;
        else {
            proton_candidates.push_back(i);
        }
        
    }
    double leading_len = -99.;
    for (auto i: proton_candidates){
        if (leading_len < trk_len_v[i]){
            leading_len = trk_len_v[i];
            leading = i;
        }
    }
    
    return leading;
}

In [19]:
%%cpp 
int select_0(Int_t slice_orig_pass_id){
    int passfail = 1;
    if (slice_orig_pass_id == 0){
        passfail = 0;
    }
    return passfail;
}

In [20]:
%%cpp
int select_1(Float_t reco_nu_vtx_sce_x, Float_t reco_nu_vtx_sce_y, Float_t reco_nu_vtx_sce_z){
    int passfail = 1;
    if (reco_nu_vtx_sce_x > 234.85 || reco_nu_vtx_sce_x < 21.5){
        passfail = 0;
    }
    if (reco_nu_vtx_sce_y > 95.00 || reco_nu_vtx_sce_y < -95.00){
        passfail = 0;
    }
    if (reco_nu_vtx_sce_z > 966.8 || reco_nu_vtx_sce_z < 21.5){
        passfail = 0;
    }
    return passfail;
}

In [21]:
%%cpp
int select_2(ROOT::VecOps::RVec<unsigned int> generation, ROOT::VecOps::RVec<float> trk_sce_start_x_v,
   ROOT::VecOps::RVec<float> trk_sce_start_y_v, ROOT::VecOps::RVec<float> trk_sce_start_z_v){
    int passfail = 1;
    for (size_t i = 0; i < generation.size(); ++i){
        if (generation[i] != 2) continue;
        if (trk_sce_start_x_v[i] > 246.35 || trk_sce_start_x_v[i] < 10) {
            passfail = 0;
        }
        if (trk_sce_start_y_v[i] > 106.50 || trk_sce_start_y_v[i] < -106.50) {
            passfail = 0;
        }
        if (trk_sce_start_z_v[i] > 1026.8 || trk_sce_start_z_v[i] < 10) {
            passfail = 0;
        }
    }
    return passfail;
}

In [22]:
%%cpp
int select_3(Float_t topological_score){
    int passfail = 1;
    if (topological_score < 0.1){
        passfail = 0;
    }
    return passfail;
}

In [23]:
%%cpp
int select_4(ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_score_v,
    ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_len_v){
        int passfail = 0;
        
        for (size_t i = 0; i < trk_llr_pid_score_v.size(); ++i){
            if ((trk_score_v[i] > 0.8)&&(trk_distance_v[i] < 4.0) && (trk_len_v[i] > 10.00)&& (trk_llr_pid_score_v[i] > 0.2)){
                passfail = 1;
            }
            
    
    }
    return passfail;
}

In [24]:
%%cpp
int select_5(ROOT::VecOps::RVec<float> trk_sce_end_x_v, ROOT::VecOps::RVec<float> trk_sce_end_y_v, ROOT::VecOps::RVec<float> trk_sce_end_z_v,
            ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_len_v, ROOT::VecOps::RVec<float> trk_score_v){
    int passfail = 1;
    ROOT::VecOps::RVec<int> muon_indices;
    for (size_t i = 0; i < trk_llr_pid_score_v.size(); ++i){
            if (trk_score_v[i] <= 0.8){
                continue;
            }
            if (trk_distance_v[i] >= 4.0){
                continue;
            }
            if (trk_len_v[i] <= 10.0){
                continue;
            }
            if (trk_llr_pid_score_v[i] <= 0.2){
                continue;
            }
            else {
                muon_indices.push_back(i);
                
            }
        }
    int muon_i = -999;
    
    if (muon_indices.size() == 0){
        passfail = 0;
        return passfail;
    }
    float muon_score = -99.0;
    if (muon_indices.size() > 1){
        for (auto i: muon_indices){
            if (trk_llr_pid_score_v[i] > muon_score) {
                muon_score = trk_llr_pid_score_v[i];
                muon_i = i;
            }
        }
    }
    else{
        muon_i = muon_indices[0];
    }
    bool out_x = trk_sce_end_x_v[muon_i] >= 246.35 || trk_sce_end_x_v[muon_i] <= 10.0;
    bool out_y = trk_sce_end_y_v[muon_i] >= 106.5 || trk_sce_end_y_v[muon_i] <= -106.5;
    bool out_z = trk_sce_end_z_v[muon_i] >= 1026.8 || trk_sce_end_z_v[muon_i] <= 10.0;
    
    if (out_x || out_y || out_z){
        passfail = 0;
    }
    
    
    return passfail;
}

In [25]:
%%cpp
int select_6(ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_len_v, ROOT::VecOps::RVec<float> trk_score_v,
            ROOT::VecOps::RVec<float> trk_range_muon_mom_v, ROOT::VecOps::RVec<float> trk_mcs_muon_mom_v){
    int passfail = 1;
    ROOT::VecOps::RVec<int> muon_indices;
    for (size_t i = 0; i < trk_llr_pid_score_v.size(); ++i){
            if (trk_score_v[i] <= 0.8){
                continue;
            }
            if (trk_distance_v[i] >= 4.0){
                continue;
            }
            if (trk_len_v[i] <= 10.0){
                continue;
            }
            if (trk_llr_pid_score_v[i] <= 0.2){
                continue;
            }
            else {
                muon_indices.push_back(i);
                
            }
        }
    int muon_i = -999;
    
    if (muon_indices.size() == 0){
        passfail = 0;
        return passfail;
    }
    float muon_score = -99.0;
    if (muon_indices.size() > 1){
        for (auto i: muon_indices){
            if (trk_llr_pid_score_v[i] > muon_score) {
                muon_score = trk_llr_pid_score_v[i];
                muon_i = i;
            }
        }
    }
    else{
        muon_i = muon_indices.at(0);
    }
    
    float percent_diff = std::abs(trk_range_muon_mom_v[muon_i] - trk_mcs_muon_mom_v[muon_i]) / trk_range_muon_mom_v[muon_i];
    if (percent_diff >= 0.25){
        passfail = 0;
    }
    
    return passfail;
    
}

In [26]:
%%cpp 
int select_7(ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_len_v, ROOT::VecOps::RVec<float> trk_score_v,
            ROOT::VecOps::RVec<double> trk_range_muon_mom_v, ROOT::VecOps::RVec<double> trk_mcs_muon_mom_v,
             ROOT::VecOps::RVec<float> trk_sce_end_x_v, ROOT::VecOps::RVec<float> trk_sce_end_y_v, ROOT::VecOps::RVec<float> trk_sce_end_z_v){
    int passfail = 1;
    int muon_i = muon_ID(trk_llr_pid_score_v, trk_distance_v,  trk_score_v,trk_len_v);
    double mom = -1.0;
    bool out_x = trk_sce_end_x_v[muon_i] >= 246.35 || trk_sce_end_x_v[muon_i] <= 10.0;
    bool out_y = trk_sce_end_y_v[muon_i] >= 106.5 || trk_sce_end_y_v[muon_i] <= -106.5;
    bool out_z = trk_sce_end_z_v[muon_i] >= 1026.8 || trk_sce_end_z_v[muon_i] <= 10.0;
    if (out_x || out_y || out_z){
        mom = trk_mcs_muon_mom_v[muon_i];
    }
    else{
        mom = trk_range_muon_mom_v[muon_i];
    }
    if (mom > 1.20 || mom < 0.1){
        passfail = 0;
    }
    return passfail;
}

In [27]:
%%cpp
int select_8(ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<int> generation_v){
    int passfail = 1;
    for (size_t i = 0; i < trk_score_v.size(); ++i){
        if (generation_v.at(i) != 2) continue;
        if (trk_score_v.at(i) <= 0.5){
            passfail = 0;
            return passfail;
        }
        
        
    }
    return passfail;
}

In [28]:
%%cpp 
int select_9(ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<int> generation_v, ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v,
            ROOT::VecOps::RVec<float> trk_len_v){
    int passfail = 1;
    vector<int> proton_candidates;
    
    int muon_i = muon_ID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v);
    
    for (size_t i = 0; i < trk_score_v.size(); ++i){
        if (generation_v[i] != 2) continue;
        if (i == muon_i) continue;
        if (trk_score_v[i] < 0.5){
           
            break;
        }
        else {
            proton_candidates.push_back(i);
        }
    }
    if (proton_candidates.size() == 0){
        passfail = 0;
    }
    return passfail;
}

In [29]:
%%cpp 
int select_10(ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<int> generation_v, ROOT::VecOps::RVec<float> trk_sce_end_x_v, 
             ROOT::VecOps::RVec<float> trk_sce_end_y_v, ROOT::VecOps::RVec<float> trk_sce_end_z_v,
             ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_len_v, ROOT::VecOps::RVec<float> trk_llr_pid_score_v){
    int passfail = 1;
    vector<int> proton_candidates;
    int muon_i = muon_ID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v);
    for (size_t i = 0; i < trk_score_v.size(); ++i){
        if (generation_v[i] != 2) continue;
        if (i == muon_i) continue;
        if (trk_score_v[i] < 0.5){
           
            break;
        }
        else {
            proton_candidates.push_back(i);
        }
    }
    for (auto i: proton_candidates){
        if (trk_sce_end_x_v[i] < 10.0 || trk_sce_end_x_v[i] > 246.35){
            passfail = 0;
            break;
        }
        if (trk_sce_end_y_v[i] < -106.5 || trk_sce_end_y_v[i] > 106.5){
            passfail = 0;
            break;
        }
        if (trk_sce_end_z_v[i] < 10.0 || trk_sce_end_z_v[i] > 1026.8){
            passfail = 0;
            break;
        }
        
    }
    return passfail;
}

In [30]:
%%cpp
int select_11(ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<int> generation_v, ROOT::VecOps::RVec<float> trk_llr_pid_score_v,
             ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_len_v){
    int passfail = 1;
    vector<int> proton_candidates;
    int muon_i = muon_ID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v);
    for (size_t i = 0; i < trk_score_v.size(); ++i){
        if (generation_v[i] != 2) continue;
        if (i == muon_i) continue;
        if (trk_score_v[i] < 0.5){
           
            break;
        }
        else {
            proton_candidates.push_back(i);
        }
    }
    for (auto i: proton_candidates){
        if (trk_llr_pid_score_v[i] > 0.2){
            passfail = 0;
            break;
        }
    }
    return passfail;
}

In [31]:
%%cpp
int select_12(ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<int> generation_v, ROOT::VecOps::RVec<float> trk_energy_proton_v,
             ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_len_v){
    int passfail = 1;
    vector<int> proton_candidates;
    int muon_i = muon_ID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v);
    for (size_t i = 0; i < trk_score_v.size(); ++i){
        if (generation_v[i] != 2) continue;
        if (i == muon_i) continue;
        if (trk_score_v[i] < 0.5) continue;
        else {
            proton_candidates.push_back(i);
        }
    }
    int leading_i = LeadingID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v, trk_energy_proton_v, generation_v);
    
    double leading_mom = rel_mom(0.93827208, trk_energy_proton_v[leading_i]);
    if (leading_mom < 0.25 || leading_mom > 1.00) {
           passfail = 0 ;
    }
     return passfail;
}

In [32]:
%%cpp
int fullSelection(Int_t slice_orig_pass_id,
             Float_t reco_nu_vtx_sce_x, 
             Float_t reco_nu_vtx_sce_y, 
             Float_t reco_nu_vtx_sce_z,
             ROOT::VecOps::RVec<float> trk_sce_start_x_v,
             ROOT::VecOps::RVec<float> trk_sce_start_y_v,
             ROOT::VecOps::RVec<float> trk_sce_start_z_v,
             Float_t topological_score,
             ROOT::VecOps::RVec<float> trk_distance_v, 
             ROOT::VecOps::RVec<float> trk_len_v,
             ROOT::VecOps::RVec<float> trk_sce_end_x_v, 
             ROOT::VecOps::RVec<float> trk_sce_end_y_v, 
             ROOT::VecOps::RVec<float> trk_sce_end_z_v, 
             ROOT::VecOps::RVec<float> trk_range_muon_mom_v, 
             ROOT::VecOps::RVec<float> trk_mcs_muon_mom_v, 
             ROOT::VecOps::RVec<float> pfp_generation_v, 
             ROOT::VecOps::RVec<float> trk_score_v, 
             ROOT::VecOps::RVec<float> trk_llr_pid_score_v, 
             ROOT::VecOps::RVec<float> trk_energy_proton_v){
    if (select_0(slice_orig_pass_id) == 0
        ||select_1(reco_nu_vtx_sce_x, reco_nu_vtx_sce_y, reco_nu_vtx_sce_z) == 0 
        || select_2(pfp_generation_v, trk_sce_start_x_v, trk_sce_start_y_v, trk_sce_start_z_v) == 0
        || select_3(topological_score) == 0
        || select_4(trk_llr_pid_score_v, trk_score_v, trk_distance_v, trk_len_v) == 0
        || select_5(trk_sce_end_x_v, trk_sce_end_y_v, trk_sce_end_z_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v, trk_score_v) == 0
        || select_6(trk_llr_pid_score_v, trk_distance_v, trk_len_v, trk_score_v, trk_range_muon_mom_v, trk_mcs_muon_mom_v) == 0
        || select_7(trk_llr_pid_score_v, trk_distance_v, trk_len_v, trk_score_v, trk_range_muon_mom_v, trk_mcs_muon_mom_v, trk_sce_end_x_v, trk_sce_end_y_v, trk_sce_end_z_v) == 0
        ||select_8(trk_score_v, pfp_generation_v) == 0
        ||select_9(trk_score_v, pfp_generation_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v) == 0
        || select_10(trk_score_v, pfp_generation_v, trk_sce_end_x_v, trk_sce_end_y_v, trk_sce_end_z_v, trk_distance_v, trk_len_v, trk_llr_pid_score_v) == 0
        || select_11(trk_score_v, pfp_generation_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v) == 0
        || select_12(trk_score_v, pfp_generation_v, trk_energy_proton_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v) == 0){
        return 0;
    }

    else{
        return 1;
    }
}


In [33]:
%%cpp
int find_category(Float_t true_nu_vtx_x, 
             Float_t true_nu_vtx_y, 
             Float_t true_nu_vtx_z,
             Int_t ccnc,
             ROOT::VecOps::RVec<int> mc_pdg, 
             ROOT::VecOps::RVec<float> mc_px, 
             ROOT::VecOps::RVec<float> mc_py, 
             ROOT::VecOps::RVec<float> mc_pz,
             int nu_pdg,
             int interaction){
    bool isSignal = true;
    int EventID = -1;
    if (is_Signal(true_nu_vtx_x, true_nu_vtx_y, true_nu_vtx_z, nu_pdg, ccnc, mc_pdg, mc_px, mc_py, mc_pz) == 0){
        isSignal = false;
    }
    if (true_nu_vtx_x > 234.85 || true_nu_vtx_x < 21.5){
            EventID = 9;
            return EventID;
        }
        else if (true_nu_vtx_y > 95.00 || true_nu_vtx_y < -95.00){
            EventID = 9;
            return EventID;        
        }
        else if (true_nu_vtx_z > 966.8 || true_nu_vtx_z < 21.5){
            EventID = 9;
            return EventID;
        }
    
    else if (ccnc == 1){
            EventID = 8;
            return EventID;
        }
    else if (ccnc == 0 && nu_pdg == 12){
            EventID = 7;
            return EventID;
        }
    else if (nu_pdg != 14){
        EventID = 10;
        return EventID;
    }
    if (isSignal){
        if (interaction == 0){
            EventID = 0;
            return EventID;
        }
        if (interaction == 10){
            EventID = 1;
            return EventID;
        }
        if (interaction == 1){
            EventID = 2;
            return EventID;
        }
        else {
            EventID = 3;
            return EventID;
        }
    }
    else{
        
        
        ROOT::VecOps::RVec<int> pions = {111, 211};
        int num_pi = 0;
        int num_pro = 0;
        if (nu_pdg == 14 && ccnc == 0){
            for (size_t i = 0; i < mc_pdg.size(); ++i){
                if (mc_pdg[i] == 2212){
                    float mag = magnitude(mc_px[i], mc_py[i], mc_pz[i]);
                    if (mag >= 0.25 && mag <= 1.0){
                        num_pro += 1;
                    }
                }
                for (size_t j = 0; j < pions.size(); ++j){
                    if (std::abs(mc_pdg[i]) == pions[j]){
                        num_pi += 1;
                    }
                }
            }
        
        }
        if (nu_pdg == 14 && ccnc == 0 && num_pi > 0){
            EventID = 4;
            return EventID;
        }
        else if (num_pi == 0 && num_pro == 0){
            EventID = 5;
            return EventID;
        }
        else {
            EventID = 6;
            return EventID;
        }
        
    }
}

In [34]:
%%cpp
float p_p(ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<float> trk_len_v,
            ROOT::VecOps::RVec<float> trk_energy_proton_v, ROOT::VecOps::RVec<int> pfp_generation_v){
    float mag = -1;
    for (size_t i = 0; i < trk_energy_proton_v.size(); ++i){
        if (i != LeadingID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v, trk_energy_proton_v, pfp_generation_v)) continue;
        mag = rel_mom(0.93827208, trk_energy_proton_v[i]);
    }
    return mag;
}

In [35]:
%%cpp
float opening_angle(ROOT::VecOps::RVec<float> trk_range_muon_mom_v, ROOT::VecOps::RVec<float> trk_mcs_muon_mom_v,
                   ROOT::VecOps::RVec<float> trk_dir_x_v, ROOT::VecOps::RVec<float> trk_dir_y_v, ROOT::VecOps::RVec<float> trk_dir_z_v,
                   ROOT::VecOps::RVec<float> trk_llr_pid_score_v, ROOT::VecOps::RVec<float> trk_distance_v, ROOT::VecOps::RVec<float> trk_score_v, ROOT::VecOps::RVec<float> trk_len_v,
                   ROOT::VecOps::RVec<float> trk_energy_proton_v, ROOT::VecOps::RVec<int> generation_v,
                   ROOT::VecOps::RVec<float> trk_sce_end_x_v, ROOT::VecOps::RVec<float> trk_sce_end_y_v, ROOT::VecOps::RVec<float> trk_sce_end_z_v){
    float u_i = muon_ID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v);
    float p_i = LeadingID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v, trk_energy_proton_v, generation_v);
    float mag_p = rel_mom(0.93827208, trk_energy_proton_v[p_i]);
    float mag_u = -1;
    float mom = -1.0;
    bool out_x = trk_sce_end_x_v[u_i] >= 246.35 || trk_sce_end_x_v[u_i] <= 10.0;
    bool out_y = trk_sce_end_y_v[u_i] >= 106.5 || trk_sce_end_y_v[u_i] <= -106.5;
    bool out_z = trk_sce_end_z_v[u_i] >= 1026.8 || trk_sce_end_z_v[u_i] <= 10.0;
    if (out_x || out_y || out_z){
        mag_u = trk_mcs_muon_mom_v[u_i];
    }
    else{
        mag_u = trk_range_muon_mom_v[u_i];
    }
    float u_x = trk_dir_x_v[u_i]* mag_u;
    float u_y = trk_dir_y_v[u_i]* mag_u;
    float u_z = trk_dir_z_v[u_i]* mag_u;
    
    float p_x = trk_dir_x_v[p_i] * mag_p;
    float p_y = trk_dir_z_v[p_i] * mag_p;
    float p_z = trk_dir_z_v[p_i] * mag_p;
    
    float dot = (u_x * p_x + u_y * p_y + u_z*p_z);
    float mag = mag_p * mag_u;
    return dot / mag;
}

In [74]:
select1 = df.Define('II', 'select_1(reco_nu_vtx_sce_x, reco_nu_vtx_sce_y, reco_nu_vtx_sce_z)')
select2 = select1.Define('III', 'select_2(pfp_generation_v, trk_sce_start_x_v, trk_sce_start_y_v, trk_sce_start_z_v)')
select3 = select2.Define('IV', 'select_3(topological_score)')
sel4 = select3.Define('V', 'select_4(trk_llr_pid_score_v, trk_score_v, trk_distance_v, trk_len_v)')
sel5 = sel4.Define('VI', 'select_5(trk_sce_end_x_v, trk_sce_end_y_v, trk_sce_end_z_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v, trk_score_v)')
sel6 = sel5.Define('VII', 'select_6(trk_llr_pid_score_v, trk_distance_v, trk_len_v, trk_score_v, trk_range_muon_mom_v, trk_mcs_muon_mom_v)')
sel7 = sel6.Define('VIII', 'select_7(trk_llr_pid_score_v, trk_distance_v, trk_len_v, trk_score_v, trk_range_muon_mom_v, trk_mcs_muon_mom_v, trk_sce_end_x_v, trk_sce_end_y_v, trk_sce_end_z_v)')
sel8 = sel7.Define('IX', 'select_8(trk_score_v, pfp_generation_v)')
sel9 = sel8.Define('X', 'select_9(trk_score_v, pfp_generation_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v)')
sel10 = sel9.Define('XI', 'select_10(trk_score_v, pfp_generation_v, trk_sce_end_x_v, trk_sce_end_y_v, trk_sce_end_z_v, trk_distance_v, trk_len_v, trk_llr_pid_score_v)')
sel11 = sel10.Define('XII', 'select_11(trk_score_v, pfp_generation_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v)')
sel12 = sel11.Define('XIII', 'select_12(trk_score_v, pfp_generation_v, trk_energy_proton_v, trk_llr_pid_score_v, trk_distance_v, trk_len_v)')
select = sel12.Define('Selected', """fullSelection(slice_orig_pass_id, reco_nu_vtx_sce_x, 
             reco_nu_vtx_sce_y, 
             reco_nu_vtx_sce_z,
             trk_sce_start_x_v,
             trk_sce_start_y_v,
             trk_sce_start_z_v,
             topological_score,
             trk_distance_v, 
             trk_len_v,
             trk_sce_end_x_v, 
             trk_sce_end_y_v, 
             trk_sce_end_z_v, 
             trk_range_muon_mom_v, 
             trk_mcs_muon_mom_v, 
             pfp_generation_v, 
             trk_score_v, 
             trk_llr_pid_score_v, 
             trk_energy_proton_v)""")
signal = select.Define('Signal', 'is_Signal(true_nu_vtx_x, true_nu_vtx_y, true_nu_vtx_z, nu_pdg, ccnc, mc_pdg, mc_px, mc_py, mc_pz)')
category = signal.Define('Category', '''find_category(true_nu_vtx_x, 
              true_nu_vtx_y, 
              true_nu_vtx_z,
              ccnc,
              mc_pdg, 
              mc_px, 
              mc_py, 
              mc_pz,
              nu_pdg,
              interaction)''')
muID = category.Define('MuIdx', 'muon_ID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v)')
P_P = muID.Define('P_P', 'p_p(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v, trk_energy_proton_v, pfp_generation_v)')
ProID = P_P.Define('LeadPIdx', 'LeadingID(trk_llr_pid_score_v, trk_distance_v, trk_score_v, trk_len_v, trk_energy_proton_v, pfp_generation_v)')

ary = ProID.AsNumpy(['run','sub', 'evt','II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'Selected', "MuIdx", 'LeadPIdx', 'P_P'])

In [75]:
ary

{'run': ndarray([20293, 20293, 20293, ..., 20568, 20568, 20568], dtype=int32),
 'sub': ndarray([ 61,  61,  61, ..., 287, 287, 287], dtype=int32),
 'evt': ndarray([ 3054,  3058,  3062, ..., 14374, 14375, 14383], dtype=int32),
 'II': ndarray([1, 0, 1, ..., 0, 1, 0], dtype=int32),
 'III': ndarray([1, 0, 1, ..., 0, 1, 1], dtype=int32),
 'IV': ndarray([1, 0, 1, ..., 1, 1, 1], dtype=int32),
 'V': ndarray([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 'VI': ndarray([0, 0, 0, ..., 1, 0, 1], dtype=int32),
 'VII': ndarray([1, 1, 0, ..., 1, 0, 1], dtype=int32),
 'VIII': ndarray([1, 1, 1, ..., 1, 0, 1], dtype=int32),
 'IX': ndarray([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 'X': ndarray([1, 0, 1, ..., 0, 1, 1], dtype=int32),
 'XI': ndarray([0, 1, 1, ..., 1, 1, 1], dtype=int32),
 'XII': ndarray([0, 1, 1, ..., 1, 1, 0], dtype=int32),
 'XIII': ndarray([1, 0, 0, ..., 0, 1, 1], dtype=int32),
 'Selected': ndarray([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'MuIdx': ndarray([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Lea

In [76]:
ProID.Snapshot('PostCuts', 'Post_Selection_Data.root', ['run','sub', 'evt','II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'Selected', 'Signal', 'Category',  "MuIdx", 'LeadPIdx', 'reco_nu_vtx_sce_x', 
             'reco_nu_vtx_sce_y', 
             'reco_nu_vtx_sce_z',
             'trk_sce_start_x_v',
             'trk_sce_start_y_v',
             'trk_sce_start_z_v',
             'topological_score',
             'trk_distance_v', 
             'trk_len_v',
             'trk_sce_end_x_v', 
             'trk_sce_end_y_v', 
             'trk_sce_end_z_v', 
             'trk_range_muon_mom_v', 
             'trk_mcs_muon_mom_v', 
             'pfp_generation_v', 
             'trk_score_v',
             'trk_llr_pid_score_v', 
             'trk_energy_proton_v', 'true_nu_vtx_x', 'true_nu_vtx_y', 'true_nu_vtx_z', 'nu_pdg', 'ccnc', 'mc_pdg', 'mc_px', 'mc_py', 'mc_pz','P_P'])

<cppyy.gbl.ROOT.RDF.RResultPtr<ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void> > object at 0x2b99bee0>

In [39]:
with open("AllCuts.txt", "w") as file:
    # Write header with exactly one space between column names
    file.write(' '.join(ary.keys()) + "\n")

    # Get number of events (rows)
    num_rows = len(next(iter(ary.values())))

    # Write each event
    for i in range(num_rows):
        # Build a list of string entries for this row
        row_entries = [str(ary[key][i]) for key in ary]
        # Join them with a single space
        line = ' '.join(row_entries)
        file.write(line+ "\n")  # Extra newline between events


In [40]:
#ProID.Snapshot('PostCuts', 'Post_Selection_Data.root', ["Category", "II", "III", "IV", "IX", "LeadPIdx", "MuIdx", "Selected", "Signal", "V", "VI", "VII", "VIII", "X", "XI", "XII", "XIII", "ccnc", "evt", "mc_pdg", "mc_px", "mc_py", "mc_pz", "nu_pdg", "pfp_generation_v", "reco_nu_vtx_sce_x", "reco_nu_vtx_sce_y", "reco_nu_vtx_sce_z", "run", "sub", "topological_score", "trk_distance_v", "trk_energy_proton_v", "trk_len_v", "trk_llr_pid_score_v", "trk_mcs_muon_mom_v", "trk_range_muon_mom_v", "trk_sce_end_x_v", "trk_sce_end_y_v", "trk_sce_end_z_v", "trk_sce_start_x_v", "trk_sce_start_y_v", "trk_sce_start_z_v", "trk_score_v", "true_nu_vtx_x", "true_nu_vtx_y", "true_nu_vtx_z", "weightSplineTimesTune"])

In [41]:
f1.Close()